In [1]:
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(25)

2025-03-30 21:49:59.965394: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 21:49:59.974215: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-30 21:49:59.996855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743367800.033907    4995 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743367800.044849    4995 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743367800.077240    4995 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Load data
csv_path = '../../datasets/human_or_ai_dataset_small.csv'  # Change this to your file path
df = pd.read_csv(csv_path)
print("Dataset shape:", df.shape)
print("Columns:", df.columns)

# Parameters
max_length = 600
max_tokens = 20000

# Extract texts and labels
texts = df['text'].values
labels = df['source'].values

# Convert labels to numeric values
label_map = {'human': 0, 'ai': 1}
y_data = np.array([label_map[label] for label in labels])

# Define TextVectorization layer
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

# Adapt to the text dataset
text_vectorization.adapt(texts)

# Transform text data into tokenized sequences
x_data = text_vectorization(texts).numpy()

# Split data
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Check shapes
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

Dataset shape: (5051, 2)
Columns: Index(['text', 'source'], dtype='object')


E0000 00:00:1743367807.397342    4995 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1743367807.398779    4995 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


x_train shape: (4040, 600)
y_train shape: (4040,)
x_test shape: (1011, 600)
y_test shape: (1011,)


## Zero-Shot Learning
Zero-Shot Learning (ZSL) é uma abordagem onde o modelo é capaz de classificar exemplos de classes que nunca viu durante o treino. Para isso, usam-se embeddings pré-treinados e é preciso ajustar o modelo para generalizar para novas classes.

In [3]:
from transformers import BartTokenizer, BartModel
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# load do modelo e tokenizer do BART
bart_model = BartModel.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# fun para obter embeddings BART
def get_bart_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bart_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Usa o embedding do primeiro token

# descrições das classes
human_description = "Text written by a human is creative and expressive."  
ai_description = "Text generated by artificial intelligence is predictable and structured."

# embeddings das classes
human_embedding = get_bart_embedding(human_description)
ai_embedding = get_bart_embedding(ai_description)

# fun para classificar um texto
def classify_text_bart(text):
    text_embedding = get_bart_embedding(text)
    
    # similaridade coseno
    human_similarity = np.dot(text_embedding, human_embedding) / (np.linalg.norm(text_embedding) * np.linalg.norm(human_embedding))
    ai_similarity = np.dot(text_embedding, ai_embedding) / (np.linalg.norm(text_embedding) * np.linalg.norm(ai_embedding))
    
    return 0 if human_similarity > ai_similarity else 1  # 0 = humano, 1 = IA

# reduzir pras x primeiras amostras
df_shorted = df.head(100)

y_true = np.array([label_map[label] for label in df_shorted['source'].values])

y_pred = [classify_text_bart(text) for text in df_shorted['text'].values]

# acc
accuracy = accuracy_score(y_true, y_pred)
print(f"acc BART: {accuracy}")


acc BART: 0.88


In [6]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# load do modelo e tokenizer do RoBERTa
roberta_model = RobertaModel.from_pretrained("roberta-base")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# fun para obter embeddings do RoBERTa
def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = roberta_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Usa o embedding do primeiro token

# obter embeddings das descricoes
human_embedding = get_roberta_embedding(human_description)
ai_embedding = get_roberta_embedding(ai_description)

# fun para classificar um texto
def classify_text_roberta(text):
    text_embedding = get_roberta_embedding(text)
    
    # similaridade coseno
    human_similarity = np.dot(text_embedding, human_embedding) / (np.linalg.norm(text_embedding) * np.linalg.norm(human_embedding))
    ai_similarity = np.dot(text_embedding, ai_embedding) / (np.linalg.norm(text_embedding) * np.linalg.norm(ai_embedding))
    
    return 0 if human_similarity > ai_similarity else 1  # 0 = humano, 1 = IA


y_true = np.array([label_map[label] for label in df_shorted['source'].values])

y_pred = [classify_text_roberta(text) for text in df_shorted['text'].values]

# acc
accuracy = accuracy_score(y_true, y_pred)
print(f"acc RoBERTa: {accuracy}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


acc RoBERTa: 0.44


In [7]:
from transformers import RobertaTokenizer, DistilBertModel
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# load do modelo e tokenizer do DistilRoBERTa
distilroberta_model = DistilBertModel.from_pretrained("distilroberta-base")
tokenizer = RobertaTokenizer.from_pretrained("distilroberta-base")

# fun para obter embeddings do DistilRoBERTa
def get_distilroberta_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = distilroberta_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Usa o embedding do primeiro token

# embeddings das descricoes
human_embedding = get_distilroberta_embedding(human_description)
ai_embedding = get_distilroberta_embedding(ai_description)

# fun para classificar um texto
def classify_text_distilroberta(text):
    text_embedding = get_distilroberta_embedding(text)
    
    # similaridade coseno
    human_similarity = np.dot(text_embedding, human_embedding) / (np.linalg.norm(text_embedding) * np.linalg.norm(human_embedding))
    ai_similarity = np.dot(text_embedding, ai_embedding) / (np.linalg.norm(text_embedding) * np.linalg.norm(ai_embedding))
    
    return 0 if human_similarity > ai_similarity else 1  # 0 = humano, 1 = IA


y_true = np.array([label_map[label] for label in df_shorted['source'].values])

y_pred = [classify_text_distilroberta(text) for text in df_shorted['text'].values]

# acc
accuracy = accuracy_score(y_true, y_pred)
print(f"acc DistilRoBERTa: {accuracy}")


You are using a model of type roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertModel were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.

acc DistilRoBERTa: 0.52


## One-Shot Learning
One-Shot Learning (OSL) é uma abordagem onde o modelo é treinado para classificar exemplos de classes com base num único exemplo de cada classe. Isto é útil quando há poucos dados disponíveis.

In [8]:
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, BartTokenizer, BartModel
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# carregar modelos e tokenizers
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

roberta_model = RobertaModel.from_pretrained("roberta-base")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

bart_model = BartModel.from_pretrained("facebook/bart-large")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# funcao para obter embeddings
def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Embedding do primeiro token


# Exemplo de One-Shot de cada classe 
human_example = df[df['source'] == 'human'].iloc[0]['text']
ai_example = df[df['source'] == 'ai'].iloc[0]['text']

# Obter embeddings de BERT, RoBERTa e BART
human_embedding_bert = get_embedding(human_example, bert_tokenizer, bert_model)
ai_embedding_bert = get_embedding(ai_example, bert_tokenizer, bert_model)

human_embedding_roberta = get_embedding(human_example, roberta_tokenizer, roberta_model)
ai_embedding_roberta = get_embedding(ai_example, roberta_tokenizer, roberta_model)

human_embedding_bart = get_embedding(human_example, bart_tokenizer, bart_model)
ai_embedding_bart = get_embedding(ai_example, bart_tokenizer, bart_model)

# Função para classificar um texto usando BERT
def classify_text_bert(text):
    text_embedding = get_embedding(text, bert_tokenizer, bert_model)
    human_similarity = np.dot(text_embedding, human_embedding_bert) / (np.linalg.norm(text_embedding) * np.linalg.norm(human_embedding_bert))
    ai_similarity = np.dot(text_embedding, ai_embedding_bert) / (np.linalg.norm(text_embedding) * np.linalg.norm(ai_embedding_bert))
    return 0 if human_similarity > ai_similarity else 1  # 0 = humano, 1 = IA

# Função para classificar um texto usando RoBERTa
def classify_text_roberta(text):
    text_embedding = get_embedding(text, roberta_tokenizer, roberta_model)
    human_similarity = np.dot(text_embedding, human_embedding_roberta) / (np.linalg.norm(text_embedding) * np.linalg.norm(human_embedding_roberta))
    ai_similarity = np.dot(text_embedding, ai_embedding_roberta) / (np.linalg.norm(text_embedding) * np.linalg.norm(ai_embedding_roberta))
    return 0 if human_similarity > ai_similarity else 1  # 0 = humano, 1 = IA

# Função para classificar um texto usando BART
def classify_text_bart(text):
    text_embedding = get_embedding(text, bart_tokenizer, bart_model)
    human_similarity = np.dot(text_embedding, human_embedding_bart) / (np.linalg.norm(text_embedding) * np.linalg.norm(human_embedding_bart))
    ai_similarity = np.dot(text_embedding, ai_embedding_bart) / (np.linalg.norm(text_embedding) * np.linalg.norm(ai_embedding_bart))
    return 0 if human_similarity > ai_similarity else 1  # 0 = humano, 1 = IA


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
df_shorted = df.head(100)

y_true = np.array([label_map[label] for label in df_shorted['source'].values])

# 3 models
y_pred_bert = [classify_text_bert(text) for text in df_shorted['text'].values]
y_pred_roberta = [classify_text_roberta(text) for text in df_shorted['text'].values]
y_pred_bart = [classify_text_bart(text) for text in df_shorted['text'].values]

# acc
accuracy_bert = accuracy_score(y_true, y_pred_bert)
accuracy_roberta = accuracy_score(y_true, y_pred_roberta)
accuracy_bart = accuracy_score(y_true, y_pred_bart)

print(f"acc BERT: {accuracy_bert}")
print(f"acc RoBERTa: {accuracy_roberta}")
print(f"acc BART: {accuracy_bart}")

acc BERT: 0.95
acc RoBERTa: 1.0
acc BART: 1.0


## Prompt Engineering
O Prompt Engineering é a prática de projetar prompts (entradas) de forma que o modelo gere saídas de alta qualidade. A ideia é fornecer ao modelo o contexto e as instruções mais claras e específicas para obter a resposta desejada.



In [21]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# BERT 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# funcao de classificacao com engenharia de prompt
def classify_with_prompt(text):
    #prompt = f"Classifique o seguinte texto como 'humano' ou 'IA': {text}"
    prompt = f"Classify the following text as 'human' or 'AI': {text}"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    
    return torch.argmax(logits, dim=-1).item()

# 100 primeiras linhas
pe_size = 100

y_true_pe = np.array([label_map[label] for label in df['source'].head(pe_size)])  # Ajustar conforme a coluna 'source' no seu dataset
y_pred_pe = [classify_with_prompt(text) for text in df['text'].head(pe_size)]  # Ajustar conforme a coluna 'text' no seu dataset

# acc
accuracy = accuracy_score(y_true_pe, y_pred_pe)
print(f"acc PE: {accuracy}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


acc PE: 0.53


## Few Shot

In [26]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# modelo e tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# engenharia de prompt e few-shot learning
def classify_with_few_shot(text, examples):
    
    prompt = "Classify the following text as 'human' or 'AI'\n" 
    
    for example in examples:
        prompt += f"Text: {example[0]}\nClass: {example[1]}\n"
    
    prompt += f"Text: {text}\nClass:"

    # Tokenizar o prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Fazer a predict
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    
    return torch.argmax(logits, dim=-1).item()


# Criar exemplos few-shot a partir do dataset
# Selecionar 3 exemplos para cada classe (humano e IA)
human_examples = df[df['source'] == 'human'].tail(3)[['text', 'source']].values.tolist()
ai_examples = df[df['source'] == 'IA'].tail(3)[['text', 'source']].values.tolist()

few_shot_examples = []
for example in human_examples:
    few_shot_examples.append((example[0], "human"))
for example in ai_examples:
    few_shot_examples.append((example[0], "AI"))

# Testar com x entradas do dataset
pe_size = 50  

y_true_pe = np.array([1 if label == 'IA' else 0 for label in df['source'].head(pe_size)])  
y_pred_pe = [classify_with_few_shot(text, few_shot_examples) for text in df['text'].head(pe_size)]  


accuracy = accuracy_score(y_true_pe, y_pred_pe)
print(f"acc Few-shot: {accuracy}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


acc Few-shot: 1.0


In [27]:
from transformers import (
    GPT2Tokenizer,
    GPT2ForSequenceClassification,
)
from sklearn.metrics import accuracy_score


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPT-2
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
gpt2_model = GPT2ForSequenceClassification.from_pretrained("gpt2-large", num_labels=2).to(device)


# Função para classificação few-shot
def classify_with_few_shot(text, examples, tokenizer, model):
    # Construir o prompt
    prompt = "Classify the following text as 'human' or 'AI'\n" 
    
    for example in examples:
        prompt += f"Text: {example[0]}\nClass: {example[1]}\n"
    
    prompt += f"Text: {text}\nClass:"

    # Tokenizar o prompt
    gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Fazer a predict
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    
    return torch.argmax(logits, dim=-1).item()

# Criar exemplos few-shot a partir do dataset
# Selecionar 3 exemplos para cada classe (humano e IA)
human_examples = df[df['source'] == 'human'].tail(3)[['text', 'source']].values.tolist()
ai_examples = df[df['source'] == 'ai'].tail(3)[['text', 'source']].values.tolist()

few_shot_examples = []
for example in human_examples:
    few_shot_examples.append((example[0], "human"))
for example in ai_examples:
    few_shot_examples.append((example[0], "AI"))

# Testar com x entradas do dataset
pe_size = 100   

y_true_pe = np.array([1 if label == 'IA' else 0 for label in df['source'].head(pe_size)])  

y_pred_gpt2 = [classify_with_few_shot(text, few_shot_examples, gpt2_tokenizer, gpt2_model) 
               for text in df['text'].head(pe_size)]

# acc
accuracy_gpt2 = accuracy_score(y_true_pe, y_pred_gpt2)
print(f"acc GPT-2: {accuracy_gpt2}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


acc GPT-2: 1.0


## RAG (Retrieval-Augmented Generation)
O RAG é uma técnica que combina recuperação de informações com geração de texto. Em vez de confiar apenas na capacidade do modelo de gerar respostas com base no seu treino, o modelo também consulta fontes externas de dados (como uma base de dados ou documentos relevantes) para melhorar a qualidade e a precisão das respostas geradas.

Não Implementado (justificação no relatório)